In [26]:
import os
import gradio as gr
from dotenv import load_dotenv
from datetime import datetime

# --- LangChain Imports ---
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.tools import Tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.utilities import GoogleSerperAPIWrapper

# --- 1. CONFIGURATION & DIAGNOSTICS ---
load_dotenv()

print("----- 🛠️ DIAGNOSTICS CHECK -----")
if not os.getenv("OPENAI_API_KEY"):
    print("❌ ERROR: OPENAI_API_KEY is missing.")
else:
    print("✅ OPENAI_API_KEY found.")

serper_api_key = os.getenv("SERPER_API_KEY")
if not serper_api_key:
    print("⚠️ WARNING: SERPER_API_KEY missing. Search tool will be disabled.")
else:
    print("✅ SERPER_API_KEY found.")
print("--------------------------------\n")

# --- 2. TOOLS SETUP ---
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

tools = []
if serper_api_key:
    search = GoogleSerperAPIWrapper()
    tools.append(Tool(name="Search", func=search.run, description="Search the web for current events/facts."))

def get_time(*args, **kwargs):
    return datetime.now().strftime("%I:%M %p on %B %d, %Y")

def safe_calculator(expression: str) -> str:
    try:
        return str(eval(expression, {"__builtins__": None}, {}))
    except Exception as e:
        return f"Math Error: {e}"

tools.append(Tool(name="Calculator", func=safe_calculator, description="Calculate math expressions."))
tools.append(Tool(name="Time", func=get_time, description="Get current time."))

# --- 3. AGENT BRAIN SETUP ---
system_prompt = """You are a friendly, intelligent AI assistant.
- Chat naturally ("Hi", "How are you") without using tools.
- ONLY use tools (Search, Math) if the user explicitly asks for facts or calculations.
- If you use a tool, explain the answer simply.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent = create_openai_tools_agent(llm, tools, prompt)

# FIX 1: Add 'return_intermediate_steps=True' to get tool usage data
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True, 
    return_intermediate_steps=True 
)

# --- 4. CORE LOGIC ---
def process_message(user_message, history, memory_state):
    if not user_message:
        return "", history, memory_state

    if history is None: history = []
    if memory_state is None: memory_state = []

    try:
        # Run Agent
        response = agent_executor.invoke({
            "input": user_message,
            "chat_history": memory_state
        })
        
        bot_response = response["output"]
        
        # FIX 2: Check for tool usage and append to response
        # 'intermediate_steps' is a list of tuples: (AgentAction, Observation)
        steps = response.get("intermediate_steps", [])
        
        if steps:
            # Extract tool names (using set to avoid duplicates like 'Search', 'Search')
            used_tools = list(set([action.tool for action, observation in steps]))
            tool_list = ", ".join(used_tools)
            
            # Format nicely with Markdown
            prefix = f"**🛠️ Used Tool:** `{tool_list}`\n\n---\n"
            bot_response = prefix + bot_response

    except Exception as e:
        bot_response = f"❌ Error: {str(e)}"

    # Update Brain
    memory_state.append(HumanMessage(content=user_message))
    memory_state.append(AIMessage(content=bot_response))

    # Update UI
    history.append({"role": "user", "content": user_message})
    history.append({"role": "assistant", "content": bot_response})

    return "", history, memory_state

# --- 5. GRADIO UI ---
with gr.Blocks() as demo:
    gr.Markdown("# Agentic AI with Tool use")
    gr.Markdown("I show you exactly which tools I use to answer your questions.")

    memory_state = gr.State([])
    chatbot = gr.Chatbot(height=450 )
    msg = gr.Textbox(label="Your Message", placeholder="Ask: 'What time is it?' or 'Calculate 25 * 4'", autofocus=True)
    
    with gr.Row():
        btn_send = gr.Button("Send", variant="primary")
        btn_clear = gr.Button("Reset Memory")

    msg.submit(process_message, inputs=[msg, chatbot, memory_state], outputs=[msg, chatbot, memory_state])
    btn_send.click(process_message, inputs=[msg, chatbot, memory_state], outputs=[msg, chatbot, memory_state])
    
    def reset(): return "", [], []
    btn_clear.click(reset, None, [msg, chatbot, memory_state])

if __name__ == "__main__":
    demo.launch(share=False)

----- 🛠️ DIAGNOSTICS CHECK -----
✅ OPENAI_API_KEY found.
✅ SERPER_API_KEY found.
--------------------------------

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.
